### Today we are going to perform the simple classification of the amazon reviews' sentiment.

### Please, download the dataset amazon_baby.csv.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')

def remove_punctuation(text):
    import string
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

baby_df = pd.read_csv('amazon_baby.csv')
baby_df.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


## Exercise 1 (data preparation)
a) Remove punctuation from reviews using the given function.   
b) Replace all missing (nan) revies with empty "" string.  
c) Drop all the entries with rating = 3, as they have neutral sentiment.   
d) Set all positive ($\geq$4) ratings to 1 and negative($\leq$2) to -1.

In [2]:
#a)

#for row in baby_df[~baby_df['review'].isna()].itertuples():
#    baby_df.loc[row.Index, 'review'] = remove_punctuation(row.review)

baby_df.loc[~(baby_df['review'].isna()), 'review'] = baby_df.loc[~(baby_df['review'].isna()), 'review'].apply(remove_punctuation)
#short test: 
baby_df["review"][4] == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'
remove_punctuation(baby_df["review"][4]) == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'

True

In [3]:
#b)

baby_df.fillna("", inplace=True)

#short test:
baby_df["review"][38] == baby_df["review"][38]

True

In [4]:
#c)
baby_df = baby_df[~(baby_df['rating']==3)]
#short test:
sum(baby_df["rating"] == 3)

0

In [5]:
#d) 
baby_df.loc[baby_df.rating < 3, 'rating'] = -1
baby_df.loc[baby_df.rating > 3,'rating'] = 1
#short test:
sum(baby_df["rating"]**2 != 1)

0

## CountVectorizer
In order to analyze strings, we need to assign them numerical values. We will use one of the simplest string representation, which transforms strings into the $n$ dimensional vectors. The number of dimensions will be the size of our dictionary, and then the values of the vector will represent the number of appereances of the given word in the sentence.

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
reviews_train_example = ["We like apples",
                   "We hate oranges",
                   "I adore bananas",
                   "We like like apples and oranges",
                   "They dislike bananas"]

X_train_example = vectorizer.fit_transform(reviews_train_example)

print(vectorizer.get_feature_names())
print(X_train_example.todense())



['adore', 'and', 'apples', 'bananas', 'dislike', 'hate', 'like', 'oranges', 'they', 'we']
[[0 0 1 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 1 0 1]
 [1 0 0 1 0 0 0 0 0 0]
 [0 1 1 0 0 0 2 1 0 1]
 [0 0 0 1 1 0 0 0 1 0]]


In [7]:
reviews_test_example = ["They like bananas",
                   "We hate oranges bananas and apples",
                   "We love bananas"] #New word!

X_test_example = vectorizer.transform(reviews_test_example)

print(X_test_example.todense())

[[0 0 0 1 0 0 1 0 1 0]
 [0 1 1 1 0 1 0 1 0 1]
 [0 0 0 1 0 0 0 0 0 1]]


We should acknowledge few facts. Firstly, CountVectorizer does not take order into account. Secondly, it ignores one-letter words (this can be changed during initialization). Finally, for test values, CountVectorizer ignores words which are not in it's dictionary.

## Exercise 2 
a) Split dataset into training and test sets.     
b) Transform reviews into vectors using CountVectorizer. 

In [8]:
#a)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(baby_df['review'], baby_df['rating'], test_size=0.33, random_state=0)

In [9]:
#b)
vectorizer = CountVectorizer()
vectorizer.fit(baby_df['review'])

X_train_vec = vectorizer.transform(X_train.values) # .astype('uint8')
X_test_vec = vectorizer.transform(X_test.values) # .astype('uint8')


In [10]:
X_test_vec

<55029x141188 sparse matrix of type '<class 'numpy.int64'>'
	with 2930167 stored elements in Compressed Sparse Row format>

## Exercise 3 
a) Train LogisticRegression model on training data (reviews processed with CountVectorizer, ratings as they were).   
b) Print 10 most positive and 10 most negative words.

In [11]:
#a)
model = LogisticRegression()
model.fit(X_train_vec, y_train)


LogisticRegression()

In [12]:
#b)
indices = np.argsort(model.coef_, axis=1)
words = np.array(vectorizer.get_feature_names())
print("The most positive: ", words[indices[0, -10:]])
print("The most negative: ", words[indices[0, :10]])
#hint: model.coef_, vectorizer.get_feature_names()

The most positive:  ['highly' 'negative' 'complaints' 'perfect' 'complaint' 'satisfied'
 'pleased' 'awesome' 'worry' 'excellent']
The most negative:  ['worst' 'disappointing' 'concept' 'poorly' 'useless' 'worthless' 'shame'
 'poor' 'horrible' 'terrible']


## Exercise 4 
a) Predict the sentiment of test data reviews.   
b) Predict the sentiment of test data reviews in terms of probability.   
c) Find five most positive and most negative reviews.   
d) Calculate the accuracy of predictions.

In [13]:
#a)
y_pred = model.predict(X_test_vec)
y_pred

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [14]:
#b)
y_pred_proba = model.predict_proba(X_test_vec)
y_pred_proba
#hint: model.predict_proba()

array([[1.44791121e-02, 9.85520888e-01],
       [2.73648401e-01, 7.26351599e-01],
       [1.70666623e-02, 9.82933338e-01],
       ...,
       [5.09766088e-02, 9.49023391e-01],
       [2.75546961e-06, 9.99997245e-01],
       [2.49800115e-02, 9.75019989e-01]])

In [15]:
#c) 
X_test_res = X_test.reset_index()
pos_indx = np.argsort(y_pred_proba[:,1])[-5:]
pos_rew = X_test_res.iloc[pos_indx]
pos_rew
#hint: use the results of b)

,index,review
27324,103186,I am reviewing these covers because reading re...
25274,88680,After reading many online reviews asking other...
1649,48158,Were keeping this stroller After much research...
23022,152013,I researched strollers for months and months b...
42809,50735,The joovy zoom 360 was the perfect solution fo...


In [16]:
pos_indx

array([27324, 25274,  1649, 23022, 42809], dtype=int64)

In [17]:
neg_indx = np.argsort(y_pred_proba[:,0])[-5:]
neg_rew = X_test_res.iloc[neg_indx]
neg_rew

,index,review
40754,92570,I would recommend in the strongest possible wa...
42135,131738,I purchased this in the black color For some ...
38899,89902,I am so incredibly disappointed with the strol...
24937,10370,This product should be in the hall of fame sol...
45356,10180,Please see my email to the companyHelloI am wr...


In [18]:
neg_indx

array([40754, 42135, 38899, 24937, 45356], dtype=int64)

In [19]:
#d)
sum(y_pred==y_test)/len(y_test)

0.9290010721619509

## Exercise 5
In this exercise we will limit the dictionary of CountVectorizer to the set of significant words, defined below.


a) Redo exercises 2-5 using limited dictionary.   
b) Check the impact of all the words from the dictionary.   
c) Compare accuracy of predictions and the time of evaluation.

In [26]:
significant_words = ['love','great','easy','old','little','perfect','loves','well','able','car','broke','less','even','waste','disappointed','work','product','money','would','return']

In [27]:
#a)
X_train, X_test, y_train, y_test = train_test_split(baby_df['review'], baby_df['rating'], test_size=0.33, random_state=0)

In [28]:
#b)

vectorizer = CountVectorizer()
vectorizer.fit(significant_words)

X_train_vec = vectorizer.transform(X_train.values)
X_test_vec = vectorizer.transform(X_test.values)

model = LogisticRegression()
model.fit(X_train_vec, y_train)

indices = np.argsort(model.coef_, axis=1)
words = np.array(vectorizer.get_feature_names())
print("The most positive: ", words[indices[0, -10:]])
print("The most negative: ", words[indices[0, :10]])

y_pred = model.predict(X_test_vec)
y_pred_proba = model.predict_proba(X_test_vec)

X_test_res = X_test.reset_index()
pos_indx = np.argsort(y_pred_proba[:,1])[-5:]
pos_rew = X_test_res.iloc[pos_indx]

neg_indx = np.argsort(y_pred_proba[:,0])[-5:]
neg_rew = X_test_res.iloc[neg_indx]

model.score(X_test_vec, y_test)

The most positive:  ['car' 'old' 'able' 'little' 'well' 'great' 'easy' 'love' 'perfect'
 'loves']
The most negative:  ['disappointed' 'return' 'waste' 'broke' 'money' 'work' 'even' 'would'
 'product' 'less']


0.8675062239909865

In [29]:
%%time
#c)
model = LogisticRegression()
model.fit(X_train_vec, y_train)
y_pred = model.predict(X_test_vec)
y_pred_proba = model.predict_proba(X_test_vec)
sum(y_pred==y_test)/len(y_pred)
#hint: %time, %timeit

Wall time: 234 ms


0.8675062239909865

In [30]:
vectorizer = CountVectorizer()
vectorizer.fit(baby_df['review'])

X_train_vec = vectorizer.transform(X_train.values) # .astype('uint8')
X_test_vec = vectorizer.transform(X_test.values) # .astype('uint8')

In [31]:
%%time
model = LogisticRegression()
model.fit(X_train_vec, y_train)
y_pred = model.predict(X_test_vec)
y_pred_proba = model.predict_proba(X_test_vec)
sum(y_pred==y_test)/len(y_pred)

Wall time: 5.63 s


0.9290010721619509